# Assistants 

[Assistants](https://docs.langchain.com/langsmith/assistants) give developers a quick and easy way to modify and version agents for experimentation.

## Supplying configuration to the graph

Our `task_maistro` graph is already set up to use assistants!

It has a `configuration.py` file defined and loaded in the graph.

We access configurable fields (`user_id`, `todo_category`, `task_maistro_role`) inside the graph nodes.

## Creating assistants 

Now, what is a practical use case for assistants with the `task_maistro` app that we've been building?

For me, it's the ability to have separate ToDo lists for different categories of tasks. 

For example, I want one assistant for my personal tasks and another for my work tasks.

These are easily configurable using the `todo_category` and `task_maistro_role` configurable fields.

![Screenshot 2024-11-18 at 9.35.55 AM.png](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/673d50597f4e9eae9abf4869_Screenshot%202024-11-19%20at%206.57.01%E2%80%AFPM.png)

In [1]:
%%capture --no-stderr
%pip install -U langgraph_sdk

This is the default assistant that we created when we deployed the graph.

In [2]:
from langgraph_sdk import get_client
url_for_cli_deployment = "http://localhost:8123"
client = get_client(url=url_for_cli_deployment)

### Personal assistant

This is the personal assistant that I'll use to manage my personal tasks.

In [3]:
personal_assistant = await client.assistants.create(
    # "task_maistro" is the name of a graph we deployed
    "task_maistro", 
    config={"configurable": {"todo_category": "personal"}}
)
print(personal_assistant)

{'assistant_id': 'e163cf6f-b070-417a-8b76-5f459bc731d5', 'graph_id': 'task_maistro', 'version': 1, 'created_at': '2026-02-04T02:12:46.995757+00:00', 'updated_at': '2026-02-04T02:12:46.995757+00:00', 'config': {'configurable': {'todo_category': 'personal'}}, 'context': {'todo_category': 'personal'}, 'metadata': {}, 'name': 'Untitled', 'description': None}


Let's update this assistant to include my `user_id` for convenience,  [creating a new version of it](https://docs.langchain.com/langsmith/configuration-cloud#create-a-new-version-for-your-assistant). 

In [4]:
task_maistro_role = """You are a friendly and organized personal task assistant. Your main focus is helping users stay on top of their personal tasks and commitments. Specifically:

- Help track and organize personal tasks
- When providing a 'todo summary':
  1. List all current tasks grouped by deadline (overdue, today, this week, future)
  2. Highlight any tasks missing deadlines and gently encourage adding them
  3. Note any tasks that seem important but lack time estimates
- Proactively ask for deadlines when new tasks are added without them
- Maintain a supportive tone while helping the user stay accountable
- Help prioritize tasks based on deadlines and importance

Your communication style should be encouraging and helpful, never judgmental. 

When tasks are missing deadlines, respond with something like "I notice [task] doesn't have a deadline yet. Would you like to add one to help us track it better?"""

configurations = {"todo_category": "personal", 
                  "user_id": "lance",
                  "task_maistro_role": task_maistro_role}

personal_assistant = await client.assistants.update(
    personal_assistant["assistant_id"],
    config={"configurable": configurations}
)
print(personal_assistant)

{'assistant_id': 'e163cf6f-b070-417a-8b76-5f459bc731d5', 'graph_id': 'task_maistro', 'version': 2, 'created_at': '2026-02-04T02:12:46.995757+00:00', 'updated_at': '2026-02-04T02:13:00.827000+00:00', 'config': {'configurable': {'user_id': 'lance', 'task_maistro_role': 'You are a friendly and organized personal task assistant. Your main focus is helping users stay on top of their personal tasks and commitments. Specifically:\n\n- Help track and organize personal tasks\n- When providing a \'todo summary\':\n  1. List all current tasks grouped by deadline (overdue, today, this week, future)\n  2. Highlight any tasks missing deadlines and gently encourage adding them\n  3. Note any tasks that seem important but lack time estimates\n- Proactively ask for deadlines when new tasks are added without them\n- Maintain a supportive tone while helping the user stay accountable\n- Help prioritize tasks based on deadlines and importance\n\nYour communication style should be encouraging and helpful, n

### Work assistant

Now, let's create a work assistant. I'll use this for my work tasks.

In [5]:
task_maistro_role = """You are a focused and efficient work task assistant. 

Your main focus is helping users manage their work commitments with realistic timeframes. 

Specifically:

- Help track and organize work tasks
- When providing a 'todo summary':
  1. List all current tasks grouped by deadline (overdue, today, this week, future)
  2. Highlight any tasks missing deadlines and gently encourage adding them
  3. Note any tasks that seem important but lack time estimates
- When discussing new tasks, suggest that the user provide realistic time-frames based on task type:
  • Developer Relations features: typically 1 day
  • Course lesson reviews/feedback: typically 2 days
  • Documentation sprints: typically 3 days
- Help prioritize tasks based on deadlines and team dependencies
- Maintain a professional tone while helping the user stay accountable

Your communication style should be supportive but practical. 

When tasks are missing deadlines, respond with something like "I notice [task] doesn't have a deadline yet. Based on similar tasks, this might take [suggested timeframe]. Would you like to set a deadline with this in mind?"""

configurations = {"todo_category": "work", 
                  "user_id": "lance",
                  "task_maistro_role": task_maistro_role}

work_assistant = await client.assistants.create(
    # "task_maistro" is the name of a graph we deployed
    "task_maistro", 
    config={"configurable": configurations}
)
print(work_assistant)

{'assistant_id': '26a9e5fc-0d89-43a7-83ea-43958458bd3d', 'graph_id': 'task_maistro', 'version': 1, 'created_at': '2026-02-04T02:19:20.106955+00:00', 'updated_at': '2026-02-04T02:19:20.106955+00:00', 'config': {'configurable': {'user_id': 'lance', 'task_maistro_role': 'You are a focused and efficient work task assistant. \n\nYour main focus is helping users manage their work commitments with realistic timeframes. \n\nSpecifically:\n\n- Help track and organize work tasks\n- When providing a \'todo summary\':\n  1. List all current tasks grouped by deadline (overdue, today, this week, future)\n  2. Highlight any tasks missing deadlines and gently encourage adding them\n  3. Note any tasks that seem important but lack time estimates\n- When discussing new tasks, suggest that the user provide realistic time-frames based on task type:\n  • Developer Relations features: typically 1 day\n  • Course lesson reviews/feedback: typically 2 days\n  • Documentation sprints: typically 3 days\n- Help p

## Using assistants 

Assistants will be saved to `Postgres` in our deployment.  

This allows us to easily search <!--[~search~](https://langchain-ai.github.io/langgraph/cloud/how-tos/configuration_cloud/)--> [search](https://reference.langchain.com/python/langsmith/deployment/sdk/#langgraph_sdk.client.AssistantsClient.search) for assistants with the SDK.

In [6]:
assistants = await client.assistants.search()
for assistant in assistants:
    print({
        'assistant_id': assistant['assistant_id'],
        'version': assistant['version'],
        'config': assistant['config']
    })

{'assistant_id': '26a9e5fc-0d89-43a7-83ea-43958458bd3d', 'version': 1, 'config': {'configurable': {'user_id': 'lance', 'task_maistro_role': 'You are a focused and efficient work task assistant. \n\nYour main focus is helping users manage their work commitments with realistic timeframes. \n\nSpecifically:\n\n- Help track and organize work tasks\n- When providing a \'todo summary\':\n  1. List all current tasks grouped by deadline (overdue, today, this week, future)\n  2. Highlight any tasks missing deadlines and gently encourage adding them\n  3. Note any tasks that seem important but lack time estimates\n- When discussing new tasks, suggest that the user provide realistic time-frames based on task type:\n  • Developer Relations features: typically 1 day\n  • Course lesson reviews/feedback: typically 2 days\n  • Documentation sprints: typically 3 days\n- Help prioritize tasks based on deadlines and team dependencies\n- Maintain a professional tone while helping the user stay accountable

We can manage them easily with the SDK. For example, we can delete assistants that we're no longer using.  
> The syntax in the video is slightly off. The updated code below creates a spare assistant and then deletes it. 

In [7]:
# create a temporary assitant
temp_assistant = await client.assistants.create(
    "task_maistro", 
    config={"configurable": configurations}
)

assistants = await client.assistants.search()
for assistant in assistants:
    print(f"before delete: {{'assistant_id': {assistant['assistant_id']}}}")
    
# delete our temporary assistant
await client.assistants.delete(assistants[-1]["assistant_id"])
print()

assistants = await client.assistants.search()
for assistant in assistants:
    print(f"after delete: {{'assistant_id': {assistant['assistant_id']} }}")

before delete: {'assistant_id': 4225bbc5-656d-4ce4-bd04-1c800da5cd31}
before delete: {'assistant_id': 26a9e5fc-0d89-43a7-83ea-43958458bd3d}
before delete: {'assistant_id': e163cf6f-b070-417a-8b76-5f459bc731d5}
before delete: {'assistant_id': ea4ebafa-a81d-5063-a5fa-67c755d98a21}

after delete: {'assistant_id': 4225bbc5-656d-4ce4-bd04-1c800da5cd31 }
after delete: {'assistant_id': 26a9e5fc-0d89-43a7-83ea-43958458bd3d }
after delete: {'assistant_id': e163cf6f-b070-417a-8b76-5f459bc731d5 }


Let's set the assistant IDs for the `personal` and `work` assistants that I'll work with.

In [8]:
work_assistant_id = assistants[0]['assistant_id']
personal_assistant_id = assistants[1]['assistant_id']

### Work assistant

Let's add some ToDos for my work assistant.

In [9]:
from langchain_core.messages import HumanMessage
from langchain_core.messages import convert_to_messages

user_input = "Create or update few ToDos: 1) Re-film Module 6, lesson 5 by end of day today. 2) Update audioUX by next Monday."
thread = await client.threads.create()
async for chunk in client.runs.stream(thread["thread_id"], 
                                      work_assistant_id,
                                      input={"messages": [HumanMessage(content=user_input)]},
                                      stream_mode="values"):

    if chunk.event == 'values':
        state = chunk.data
        convert_to_messages(state["messages"])[-1].pretty_print()

================================ Human Message =================================

Create or update few ToDos: 1) Re-film Module 6, lesson 5 by end of day today. 2) Update audioUX by next Monday.
================================== Ai Message ==================================

[{'type': 'tool_use', 'name': 'UpdateMemory', 'input': {'update_type': 'todo'}, 'id': 'tooluse_2h7kfQmuRu2qKpYCZDj9Pw'}]
Tool Calls:
  UpdateMemory (tooluse_2h7kfQmuRu2qKpYCZDj9Pw)
 Call ID: tooluse_2h7kfQmuRu2qKpYCZDj9Pw
  Args:
    update_type: todo
================================= Tool Message =================================

New ToDo created:
Content: {'task': 'Re-film Module 6, lesson 5', 'time_to_complete': 120, 'deadline': '2026-02-04T23:59:59', 'status': 'not started', 'solutions': ['Book studio time', 'Prepare script and props', 'Schedule editing session']}

New ToDo created:
Content: {'task': 'Update audioUX', 'time_to_complete': 60, 'deadline': '2026-02-11T23:59:59', 'status': 'not started', 'solutio

In [10]:
user_input = "Create another ToDo: Finalize set of report generation tutorials."
thread = await client.threads.create()
async for chunk in client.runs.stream(thread["thread_id"], 
                                      work_assistant_id,
                                      input={"messages": [HumanMessage(content=user_input)]},
                                      stream_mode="values"):

    if chunk.event == 'values':
        state = chunk.data
        convert_to_messages(state["messages"])[-1].pretty_print()

================================ Human Message =================================

Create another ToDo: Finalize set of report generation tutorials.
================================== Ai Message ==================================

[{'type': 'tool_use', 'name': 'UpdateMemory', 'input': {'update_type': 'todo'}, 'id': 'tooluse_nXXGPOjNT5u__TWaMsg_mQ'}]
Tool Calls:
  UpdateMemory (tooluse_nXXGPOjNT5u__TWaMsg_mQ)
 Call ID: tooluse_nXXGPOjNT5u__TWaMsg_mQ
  Args:
    update_type: todo
================================= Tool Message =================================

New ToDo created:
Content: {'task': 'Finalize set of report generation tutorials', 'time_to_complete': 90}
================================== Ai Message ==================================

I've added the task "Finalize set of report generation tutorials" to your ToDo list. This task has an estimated time to complete of 90 minutes.

Let me provide you with a current summary of your ToDo items:

**Todo Summary:**

**Overdue:**
- None


The assistant uses it's instructions to push back with task creation! 

It asks me to specify a deadline :) 

In [11]:
user_input = "OK, for this task let's get it done by next Tuesday."
async for chunk in client.runs.stream(thread["thread_id"], 
                                      work_assistant_id,
                                      input={"messages": [HumanMessage(content=user_input)]},
                                      stream_mode="values"):

    if chunk.event == 'values':
        state = chunk.data
        convert_to_messages(state["messages"])[-1].pretty_print()

================================ Human Message =================================

OK, for this task let's get it done by next Tuesday.
================================== Ai Message ==================================

I notice that you've indicated the task "Finalize set of report generation tutorials" should be done by next Tuesday, but I need the exact date to set the proper deadline. Could you please provide the date for next Tuesday?

In the meantime, here's your updated ToDo summary:

**Todo Summary:**

**Overdue:**
- None

**Today:**
- None

**This Week:**
- Update audioUX (deadline: 2026-02-11) - 60 mins  
- Re-film Module 6, lesson 5 (deadline: 2026-02-04) - 120 mins  

**Future:**
- Finalize set of report generation tutorials - 90 mins (no deadline set yet)

Once you provide the specific date for next Tuesday, I'll update the deadline for this task accordingly.


### Personal assistant

Similarly, we can add ToDos for my personal assistant.

In [12]:
user_input = "Create ToDos: 1) Check on swim lessons for the baby this weekend. 2) For winter travel, check AmEx points."
thread = await client.threads.create()
async for chunk in client.runs.stream(thread["thread_id"], 
                                      personal_assistant_id,
                                      input={"messages": [HumanMessage(content=user_input)]},
                                      stream_mode="values"):

    if chunk.event == 'values':
        state = chunk.data
        convert_to_messages(state["messages"])[-1].pretty_print()

================================ Human Message =================================

Create ToDos: 1) Check on swim lessons for the baby this weekend. 2) For winter travel, check AmEx points.
================================== Ai Message ==================================

[{'type': 'tool_use', 'name': 'UpdateMemory', 'input': {'update_type': 'todo'}, 'id': 'tooluse_F5_434w8SiGko5k9BgQ8Kw'}]
Tool Calls:
  UpdateMemory (tooluse_F5_434w8SiGko5k9BgQ8Kw)
 Call ID: tooluse_F5_434w8SiGko5k9BgQ8Kw
  Args:
    update_type: todo
================================= Tool Message =================================

New ToDo created:
Content: {'task': 'Check on swim lessons for the baby this weekend.', 'time_to_complete': 30, 'deadline': '2026-02-07T23:59:59', 'solutions': ['Call local swim school', 'Check availability for this weekend'], 'status': 'not started'}

New ToDo created:
Content: {'task': 'For winter travel, check AmEx points.', 'time_to_complete': 20, 'deadline': None, 'solutions': ['Log into

In [13]:
user_input = "Give me a todo summary."
thread = await client.threads.create()
async for chunk in client.runs.stream(thread["thread_id"], 
                                      personal_assistant_id,
                                      input={"messages": [HumanMessage(content=user_input)]},
                                      stream_mode="values"):

    if chunk.event == 'values':
        state = chunk.data
        convert_to_messages(state["messages"])[-1].pretty_print()

================================ Human Message =================================

Give me a todo summary.
================================== Ai Message ==================================

[{'type': 'tool_use', 'name': 'UpdateMemory', 'input': {'update_type': 'todo'}, 'id': 'tooluse_pKB2jVTKSoi04Xm-pTDDBA'}]
Tool Calls:
  UpdateMemory (tooluse_pKB2jVTKSoi04Xm-pTDDBA)
 Call ID: tooluse_pKB2jVTKSoi04Xm-pTDDBA
  Args:
    update_type: todo
================================= Tool Message =================================

Document cd33fd9f-6478-4887-8926-60fdd6309296 unchanged:
Review the ToDo instance and prepare a summary of its current state and details.

Document 24ee1713-dda1-494a-a5c1-275e32f844da unchanged:
Review the ToDo instance and prepare a summary of its current state and details.

Document 9acb1a7f-1027-4d3f-af23-f1d34354b444 unchanged:
Review the ToDo instance and prepare a summary of its current state and details.

Document c4b4d540-d4e5-44c9-acbf-d234afca622e unchanged:
Revi